In [1]:
import os
import re
import string
from nltk.corpus import stopwords
from nltk.tokenize import sent_tokenize
from nltk.tokenize import TreebankWordTokenizer
from nltk.stem import WordNetLemmatizer
import nltk
from nltk.stem import PorterStemmer
from nltk.stem import LancasterStemmer
import numpy as np

In [2]:
yourpath = './20_newsgroups/'
all_files = []
for root, dirs, files in os.walk(yourpath, topdown=False):
    for name in files:
        all_files.append(os.path.join(root, name))

In [3]:
all_files

['./20_newsgroups/20_newsgroups\\alt.atheism\\49960',
 './20_newsgroups/20_newsgroups\\alt.atheism\\51060',
 './20_newsgroups/20_newsgroups\\alt.atheism\\51119',
 './20_newsgroups/20_newsgroups\\alt.atheism\\51120',
 './20_newsgroups/20_newsgroups\\alt.atheism\\51121',
 './20_newsgroups/20_newsgroups\\alt.atheism\\51122',
 './20_newsgroups/20_newsgroups\\alt.atheism\\51123',
 './20_newsgroups/20_newsgroups\\alt.atheism\\51124',
 './20_newsgroups/20_newsgroups\\alt.atheism\\51125',
 './20_newsgroups/20_newsgroups\\alt.atheism\\51126',
 './20_newsgroups/20_newsgroups\\alt.atheism\\51127',
 './20_newsgroups/20_newsgroups\\alt.atheism\\51128',
 './20_newsgroups/20_newsgroups\\alt.atheism\\51129',
 './20_newsgroups/20_newsgroups\\alt.atheism\\51130',
 './20_newsgroups/20_newsgroups\\alt.atheism\\51131',
 './20_newsgroups/20_newsgroups\\alt.atheism\\51132',
 './20_newsgroups/20_newsgroups\\alt.atheism\\51133',
 './20_newsgroups/20_newsgroups\\alt.atheism\\51134',
 './20_newsgroups/20_newsgro

In [3]:
Dict = {}
for i in range(len(all_files)):
    file = open(all_files[i], "r") 
    text = file.readlines()
    punctuations = '''!()-[]{};:'"\,<>./?@#$%^&*_~'''
    stop_words = set(stopwords.words('english'))
    for s in text:
        s_sep = ""
        # 4th step
        for character in s:
            if character in punctuations:
                outputCharacter = " %s " % character
            else:
                outputCharacter = character
            s_sep += outputCharacter
        # Tokenize sentences to word tokens
        for wrd in TreebankWordTokenizer().tokenize(s_sep):
            # Removed stop words and punctations
            if wrd not in punctuations:
                #6. Converted all the words to lower case
                wrd_lower = wrd.lower()
                #wrd_lower = lancaster.stem(wrd_lower)
                if ((wrd_lower not in stop_words) & (not wrd_lower.isdigit()) & (len(wrd_lower) > 2)):
                    if wrd_lower in Dict.keys():
                        if os.path.basename(all_files[i]) not in Dict[wrd_lower]:
                            Dict[wrd_lower].append(os.path.basename(all_files[i]))
                    else:
                        Dict[wrd_lower]=[]
                        Dict[wrd_lower].append(os.path.basename(all_files[i]))

In [4]:
import pickle
f = open("Dictionary_words.pkl","wb")
pickle.dump(Dict,f)
f.close()

In [15]:
import pickle 

sample_space_of_files = set()
words = dict() 

def apply_operator(list1, list2, op):
    if op == "AND":
        # x AND y
        return list(set(list1) & set(list2)) 
    elif op == "OR" :
        # x OR Y
        return list(set(list1) | set(list2))
    elif op == "AND NOT" :
        # x AND NOT Y
        return list(set(list1) & (set(sample_space_of_files) - set(list2)))
    elif op == "OR NOT" :
        # x OR NOT Y
        return list(set(list1) | (set(sample_space_of_files) - set(list2)))

# def is_operator(op):
    # return op in ["AND", "OR", "NOT"]

def get_list(word):
    return words[word]


def operation_internal(words):
    n = len(words)
    res = get_list(words[0])
    idx = 1
    while (idx + 1) < n :
        operator = words[idx]
        idx += 1
        if words[idx] == "NOT" :
            operator += " NOT"
            idx += 1
        op2 = words[idx]
        idx += 1
        res = apply_operator(res, get_list(op2), operator)

    return res 



def operation(op) :
    words = op.split()
    files = operation_internal(words)
    print (files)
    print(len(files))
    return files 



def load_dictionary(filename):
    global words 
    global sample_space_of_files

    with open(filename, 'rb') as f :
        words = pickle.load(f)

    for file_ids in words.values():
        for file in file_ids:
            sample_space_of_files.add(file)

load_dictionary('Dictionary_words.pkl')

word1 = 'cantaloupe'
word2 = 'canteens'
word3 = 'canopy'
#print(len(words))
#print(get_list(word1))
#print(get_list(word2))
files = operation(word1 + ' AND ' + word2 + ' OR ' + word3)
#cantaloupe AND canteens OR canopy

['61049', '104547', '54341']
3
